# Huggingface and W-LDA
- ArrowDataset으로 porting을 편하게!
- load_dataset을 wikitext-103을 받아오며 뜯어보자

## load_dataset
``load_dataset``은 아래 5단계를 수행한다.

1. Dataset Preprocessing Script 다운로드
2. Preprocessing Script에서 Dataset Builder Class 얻기
3. DatasetBuilder를 객체화
4. Data Download
5. Dataset Build

## (1) Dataset Preprocessing Script
- 무조건 script를 만들 것이기 때문에 다운로드할 필요없음
- 즉, 필요한 부분만 꺼내올 것
- huggingface에서 내부적으로 처리하는 코드는 아래와 같음

```python
from datasets import prepare_module


module_path, hash, resolved_file_path = prepare_module(
    path,
    script_version=script_version,
    download_config=download_config,
    download_mode=download_mode,
    dataset=True,
    return_resolved_file_path=True,
    use_auth_token=use_auth_token,
)
```

- 동작 코드 확인해보니, 아래 역할을 수행
    1. dataset인지 metric인지
    2. 강제 저장 위치 지정했는지, dynamic하게 저장할 것인지
    3. local에서 script를 가져올 것인지 아니면 github에서 가져올 것인지 결정
        - 내 코드에선 git에서 가져오는 것은 아직 불필요
        - 다음에 한 번 따로 짜보자
    4. info 파일이 있다면 다운로드

In [1]:
import os
import copy
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, Callable, List, Union, Any, NewType, Iterable, Dict

In [2]:
path = "../wikitext-103.py"
name = "wikitext-103-v1"
data_dir = None
data_files = None
split = "train"
cache_dir = "cache"
features = None
download_config = None
download_mode = None
ignore_verifications = False
keep_in_memory = False
save_infos = False
script_version = None
use_auth_token = None
config_kwargs = {}

In [3]:
@dataclass
class DownloadConfig:
    cache_dir: Optional[Union[str, Path]] = None
    force_download: bool = False
    resume_download: bool = False
    local_files_only: bool = False
    proxies: Optional[Dict] = None
    user_agent: Optional[str] = None
    extract_compressed_file: bool = False
    force_extract: bool = False
    use_etag: bool = True
    num_proc: Optional[int] = None
    max_retries: int = 1
    use_auth_token: Optional[Union[str, bool]] = None

    def copy(self) -> "DownloadConfig":
        return self.__class__(**{k: copy.deepcopy(v) for k, v in self.__dict__.items()})

In [4]:
download_config = DownloadConfig()
download_config.extract_compressed_file = True
download_config.force_extract = True
download_config

DownloadConfig(cache_dir=None, force_download=False, resume_download=False, local_files_only=False, proxies=None, user_agent=None, extract_compressed_file=True, force_extract=True, use_etag=True, num_proc=None, max_retries=1, use_auth_token=None)

In [5]:
module_type = "dataset"
_name = list(filter(lambda x: x, path.replace(os.sep, "/").split("/")))[-1]
if not _name.endswith(".py"):
    _name = _name + ".py"
    
# Short name is name without the '.py' at the end (for the module)
short_name = name[:-3]

_name, short_name

('wikitext-103.py', 'wikitext-103')

In [6]:
# first check if the module is packaged with the `datasets` package
# What is this?
from datasets.packaged_modules import _PACKAGED_DATASETS_MODULES

_PACKAGED_DATASETS_MODULES, path in _PACKAGED_DATASETS_MODULES

({'csv': ('datasets.packaged_modules.csv.csv',
   '965b6429be0fc05f975b608ce64e1fa941cc8fb4f30629b523d2390f3c0e1a93'),
  'json': ('datasets.packaged_modules.json.json',
   '5068e8663a7669137d288ea22cd76c5f4cac5f20db7ed8252b722e51c43c0760'),
  'pandas': ('datasets.packaged_modules.pandas.pandas',
   '3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791'),
  'text': ('datasets.packaged_modules.text.text',
   '44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691')},
 False)

In [7]:
dynamic_modules_path = None

In [8]:
MODULE_NAME_FOR_DYNAMIC_MODULES = "datasets_modules"

In [9]:
from datasets import config
from datasets.load import init_dynamic_modules

In [10]:
MODULE_NAME_FOR_DYNAMIC_MODULES

'datasets_modules'

In [11]:
config.HF_MODULES_CACHE

WindowsPath('C:/Users/jinma/.cache/huggingface/modules')

In [12]:
DEFAULT_XDG_CACHE_HOME = "~/.cache"
XDG_CACHE_HOME = os.getenv("XDG_CACHE_HOME", DEFAULT_XDG_CACHE_HOME)
DEFAULT_MY_CACHE_HOME = os.path.join(XDG_CACHE_HOME, "w-lda")
MY_CACHE_HOME = os.path.expanduser(os.getenv("MY_HOME", DEFAULT_MY_CACHE_HOME))
DEFAULT_MODULES_CACHE = os.path.join(MY_CACHE_HOME, "modules")
print(os.getenv("MODULES_CACHE", DEFAULT_MODULES_CACHE))
MODULES_CACHE = Path(os.getenv("MODULES_CACHE", DEFAULT_MODULES_CACHE))
MODULES_CACHE

C:\Users\jinma/.cache\w-lda\modules


WindowsPath('C:/Users/jinma/.cache/w-lda/modules')

In [13]:
# otherwise the module is added to the dynamic modules
dynamic_modules_path = (
    dynamic_modules_path
    if dynamic_modules_path is not None
    else init_dynamic_modules(MODULE_NAME_FOR_DYNAMIC_MODULES, hf_modules_cache=config.HF_MODULES_CACHE)
)

In [14]:
dynamic_modules_path

'C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules'

- 여기 필요없넹.

- dynamic_modules_path가 안주어지면 강제적으로 .cache/huggingface/modules로 설정됨
- 난 여길 항상 root direct으로 주도록 setting하면 되겠지

In [15]:
module_name_for_dynamic_modules = os.path.basename(dynamic_modules_path)
datasets_modules_path = os.path.join(dynamic_modules_path, "datasets")
datasets_modules_name = module_name_for_dynamic_modules + ".datasets"
metrics_modules_path = os.path.join(dynamic_modules_path, "metrics")
metrics_modules_name = module_name_for_dynamic_modules + ".metrics"

In [16]:
module_name_for_dynamic_modules

'datasets_modules'

In [17]:
datasets_modules_path

'C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules\\datasets'

In [18]:
datasets_modules_name

'datasets_modules.datasets'

In [19]:
metrics_modules_path

'C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules\\metrics'

In [20]:
metrics_modules_name

'datasets_modules.metrics'

In [21]:
force_local_path = None

main_folder_path = os.path.join(datasets_modules_path if True else metrics_modules_path, short_name)
main_folder_path

'C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules\\datasets\\wikitext-103'

- 여기의 if/elif/else절에서 local에 py파일이 있는지,
- 없다면 가져오는 부분이 try/except! (github에서 가져옴)
    - 이 부분은 불필요

In [22]:
combined_path = os.path.join(path, name)
combined_path

'../wikitext-103.py\\wikitext-103-v1'

In [23]:
os.path.isfile(combined_path)

False

In [24]:
path

'../wikitext-103.py'

In [25]:
os.path.isfile(path)

True

In [26]:
file_path = path
local_path = path

In [27]:
from datasets.utils.file_utils import (
    url_or_path_parent, url_or_path_join, cached_path)

In [28]:
from datasets.info import DATASET_INFOS_DICT_FILE_NAME

In [29]:
# Load the module in two steps
# 1. get the processing file on the local filesystem if it's not ther (download to cache dir)
# 2. copy from the local file system inside the modules cache to import it

base_path = url_or_path_parent(file_path)  # remove the filename
dataset_infos = url_or_path_join(base_path, DATASET_INFOS_DICT_FILE_NAME)

In [30]:
base_path, dataset_infos

('..', '../dataset_infos.json')

In [31]:
# Download the dataset infos file if available
try:
    local_dataset_infos_path = cached_path(
        dataset_infos,
        download_config=download_config,
    )
except (FileNotFoundError, ConnectionError):
    local_dataset_infos_path = None
    

local_dataset_infos_path is None

True

In [32]:
from datasets.load import get_imports # 굉장히 유용한 함수!

In [33]:
# Download external imports if needed
imports = get_imports(local_path)
local_path, imports # import_type, import_name, import_path, sub_directory

('../wikitext-103.py',
 [('library', '__future__', '__future__', None),
  ('library', 'os', 'os', None),
  ('library', 'datasets', 'datasets', None)])

In [34]:
short_name

'wikitext-103'

In [35]:
local_imports = []
library_imports = []

for _, import_name, import_path, _ in imports:
    library_imports.append((import_name, import_path)) # Import from a library

In [36]:
# Check library imports
import importlib
importlib.import_module(library_imports[0][0])
importlib.import_module(library_imports[1][0])
importlib.import_module(library_imports[2][0])
# Error X

<module 'datasets' from 'C:\\Users\\jinma\\AppData\\Local\\Continuum\\anaconda3\\envs\\basic\\lib\\site-packages\\datasets\\__init__.py'>

In [37]:
from datasets.load import files_to_hash

In [38]:
# Define a directory with a unique name in our dataset or metric folder
# path is: ./datasets|metrics/dataset|metric_name/hash_from_code/script.py
# we use a hash to be able to have multiple versions of a dataset/metric processing file together
hash_ = files_to_hash([local_path] + [loc[1] for loc in local_imports])
hash_

'6efa45b4d3a90f1e7df0212a0eeb0adfd7e96468534172fb2ec8d338fe5f6d58'

In [39]:
[local_path] + [loc[1] for loc in local_imports]

['../wikitext-103.py']

In [40]:
# 여기서 조합되는구만
hash_folder_path = os.path.join(main_folder_path, hash_)
hash_folder_path

'C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules\\datasets\\wikitext-103\\6efa45b4d3a90f1e7df0212a0eeb0adfd7e96468534172fb2ec8d338fe5f6d58'

In [41]:
local_file_path = os.path.join(hash_folder_path, name)
dataset_infos_path = os.path.join(hash_folder_path, DATASET_INFOS_DICT_FILE_NAME)

local_file_path, dataset_infos_path

('C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules\\datasets\\wikitext-103\\6efa45b4d3a90f1e7df0212a0eeb0adfd7e96468534172fb2ec8d338fe5f6d58\\wikitext-103-v1',
 'C:\\Users\\jinma\\.cache\\huggingface\\modules\\datasets_modules\\datasets\\wikitext-103\\6efa45b4d3a90f1e7df0212a0eeb0adfd7e96468534172fb2ec8d338fe5f6d58\\dataset_infos.json')

In [42]:
# Prevent parallel disk operations
lock_path = local_path + ".lock"
lock_path

'../wikitext-103.py.lock'

In [43]:
from datasets.utils.filelock import FileLock


filelock = FileLock(lock_path)

In [44]:
filelock.acquire() # ENTER

In [45]:
# Create main dataset/metrics folder if needed
from datasets.utils.download_manager import GenerateMode

# 삭제 조건이 여기에 이렇게 있네? 참고
print(main_folder_path)
if (download_mode == GenerateMode.FORCE_REDOWNLOAD and 
    os.path.exists(main_folder_path)):
    shutil.rmtree(main_folder_path)

C:\Users\jinma\.cache\huggingface\modules\datasets_modules\datasets\wikitext-103


In [46]:
from datasets.utils.logging import get_logger


logger = get_logger(__name__)

In [47]:
import logging

In [48]:
logger.setLevel(logging.INFO)

In [49]:
not os.path.exists(main_folder_path)

False

In [50]:
# 파일 없으면 만들기
if not os.path.exists(main_folder_path):
    logger.info(f"Creating main folder for {module_type} {file_path} at {main_folder_path}")
    os.makedirs(main_folder_path, exist_ok=True)
else:
    logger.info(f"Found main folder for {module_type} {file_path} at {main_folder_path}")

In [51]:
# add an __init__ file to the main dataset folder if needed
init_file_path = os.path.join(main_folder_path, "__init__.py")
print(init_file_path)
if not os.path.exists(init_file_path):
    with open(init_file_path, "w"):
        pass

C:\Users\jinma\.cache\huggingface\modules\datasets_modules\datasets\wikitext-103\__init__.py


In [114]:
# Create hash dataset folder if needed
print(not os.path.exists(hash_folder_path))
if not os.path.exists(hash_folder_path):
    logger.info(f"Creating specific version folder for {module_type} {file_path} at {hash_folder_path}")
    os.makedirs(hash_folder_path)
else:
    logger.info(f"Found specific version folder for {module_type} {file_path} at {hash_folder_path}")

False


In [115]:
# add an __init__ file to the hash dataset folder if needed
init_file_path = os.path.join(hash_folder_path, "__init__.py")
print(init_file_path)
if not os.path.exists(init_file_path):
    with open(init_file_path, "w"):
        pass

C:\Users\jinma\.cache\huggingface\modules\datasets_modules\datasets\wikitext\041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638\__init__.py


- Script 복사
- 난 굳이 할 필요 없을 듯

In [116]:
# Copy dataset.py file in hash folder if needed
print(not os.path.exists(local_file_path))
if not os.path.exists(local_file_path):
    logger.info("Copying script file from %s to %s", file_path, local_file_path)
    shutil.copyfile(local_path, local_file_path)
else:
    logger.info("Found script file from %s to %s", file_path, local_file_path)

False


- dataset info 복사
- 난 굳이...?

In [119]:
# Copy dataset infos file if needed
print(not os.path.exists(dataset_infos_path))
print(local_dataset_infos_path is not None)
print(
    local_dataset_infos_path is not None and 
    not filecmp.cmp(local_dataset_infos_path, dataset_infos_path)
)
if not os.path.exists(dataset_infos_path):
    if local_dataset_infos_path is not None:
        logger.info("Copying dataset infos file from %s to %s", dataset_infos, dataset_infos_path)
        shutil.copyfile(local_dataset_infos_path, dataset_infos_path)
    else:
        logger.info("Couldn't find dataset infos file at %s", dataset_infos)
else:
    if local_dataset_infos_path is not None and not filecmp.cmp(local_dataset_infos_path, dataset_infos_path):
        logger.info("Updating dataset infos file from %s to %s", dataset_infos, dataset_infos_path)
        shutil.copyfile(local_dataset_infos_path, dataset_infos_path)
    else:
        logger.info("Found dataset infos file from %s to %s", dataset_infos, dataset_infos_path)

True
False
False


In [120]:
# Record metadata associating original dataset path with local unique folder
meta_path = local_file_path.split(".py")[0] + ".json"
print(meta_path)
print(not os.path.exists(meta_path))
if not os.path.exists(meta_path):
    logger.info(f"Creating metadata file for {module_type} {file_path} at {meta_path}")
    meta = {"original file path": file_path, "local file path": local_file_path}
    # the filename is *.py in our case, so better rename to filenam.json instead of filename.py.json
    with open(meta_path, "w", encoding="utf-8") as meta_file:
        json.dump(meta, meta_file)
else:
    logger.info(f"Found metadata file for {module_type} {file_path} at {meta_path}")

C:\Users\jinma\.cache\huggingface\modules\datasets_modules\datasets\wikitext\041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638\wikitext.json
False


In [122]:
# Copy all the additional imports
local_imports # internal, external 정보
for import_name, import_path in local_imports:
    if os.path.isfile(import_path):
        full_path_local_import = os.path.join(hash_folder_path, import_name + ".py")
        if not os.path.exists(full_path_local_import):
            logger.info("Copying local import file from %s at %s", import_path, full_path_local_import)
            shutil.copyfile(import_path, full_path_local_import)
        else:
            logger.info("Found local import file from %s at %s", import_path, full_path_local_import)
    elif os.path.isdir(import_path):
        full_path_local_import = os.path.join(hash_folder_path, import_name)
        if not os.path.exists(full_path_local_import):
            logger.info("Copying local import directory from %s at %s", import_path, full_path_local_import)
            shutil.copytree(import_path, full_path_local_import)
        else:
            logger.info("Found local import directory from %s at %s", import_path, full_path_local_import)
    else:
        raise OSError(f"Error with local import at {import_path}")

In [126]:
filelock.release() # EXIT

In [127]:
module_path = ".".join(
    [datasets_modules_name if True else metrics_modules_name, short_name, hash_, short_name]
)
module_path

'datasets_modules.datasets.wikitext.041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638.wikitext'

In [128]:
# make the new module to be noticed by the import system
importlib.invalidate_caches()

In [130]:
return_resolved_file_path = True

In [131]:
module_path

'datasets_modules.datasets.wikitext.041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638.wikitext'

In [132]:
hash_

'041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638'

In [133]:
file_path # 맨 처음에 인자로 받은!

'wikitext.py'

## (2) Processing Script에서 Dataset Builder Class 얻기

In [171]:
from datasets.load import import_main_class

In [173]:
builder_cls = import_main_class(module_path, dataset=True)
builder_cls

datasets_modules.datasets.wikitext.041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638.wikitext.Wikitext

## (3) DatasetBuilder를 객체화

In [180]:
from datasets.builder import DatasetBuilder

In [183]:
builder_instance: DatasetBuilder = builder_cls(
    cache_dir=cache_dir,
    name=name,
    data_dir=data_dir,
    data_files=data_files,
    hash=hash_,
    features=features,
    **config_kwargs,
)

In [188]:
builder_instance

## (4) Download Data

In [190]:
# Download and prepare data
builder_instance.download_and_prepare(
    download_config=download_config,
    download_mode=download_mode,
    ignore_verifications=ignore_verifications,
    try_from_hf_gcs=True,
    base_path=base_path,
    use_auth_token=use_auth_token,
)

Dataset wikitext downloaded and prepared to cache\wikitext\wikitext-103-v1\1.0.0\041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638. Subsequent calls will reuse this data.


## (5) Dataset Build!
- 여기선 여기에 집중할 것

In [191]:
builder_instance

In [196]:
not os.path.exists(builder_instance._cache_dir)

False

In [199]:
builder_instance.info.splits

{'test': SplitInfo(name='test', num_bytes=1295579, num_examples=4358, dataset_name='wikitext'),
 'train': SplitInfo(name='train', num_bytes=545142639, num_examples=1801350, dataset_name='wikitext'),
 'validation': SplitInfo(name='validation', num_bytes=1154755, num_examples=3760, dataset_name='wikitext')}

In [204]:
from functools import partial


function = partial(
    builder_instance._build_single_dataset,
    run_post_process=True,
    ignore_verifications=False,
    in_memory=False
)
data_struct = {s: s for s in builder_instance.info.splits}
types = None
num_proc = None
map_numpy = False
map_tuple = True
map_list = False
dict_only = False

In [205]:
types = [tuple]
types = tuple(types)
types

(tuple,)

In [208]:
not isinstance(data_struct, dict)

False

In [211]:
iterable = list(data_struct.values()) if isinstance(data_struct, dict) else data_struct
iterable

['test', 'train', 'validation']

In [212]:
num_proc = 1

In [215]:
split_kwds = [] # We organize the splits ourselves (contiguous splits)
for index in range(num_proc):
    div = len(iterable) // num_proc
    mod = len(iterable) % num_proc
    start = div * index + min(index, mod)
    end = start + div + (1 if index < mod else 0)
    split_kwds.append((function, iterable[start:end], types, index, False))
assert len(iterable) == sum(len(i[1]) for i in split_kwds), (
    f"Error dividing inputs iterable among processes. "
    f"Total number of objects {len(iterable)}, "
    f"length: {sum(len(i[1]) for i in split_kwds)}"
)
div, mod, start, end, split_kwds

(3,
 0,
 0,
 3,
 [(functools.partial(<bound method DatasetBuilder._build_single_dataset of <datasets_modules.datasets.wikitext.041b2070e5d6463bbec9a8ae743190512269d9ab8aa53d33ed10a56c8a804638.wikitext.Wikitext object at 0x000002A5CB784518>>, run_post_process=True, ignore_verifications=False, in_memory=False),
   ['test', 'train', 'validation'],
   (tuple,),
   0,
   False)])

In [221]:
function, data_struct, types, rank, disable_tqdm = split_kwds[0]

In [226]:
not isinstance(data_struct, dict) and not isinstance(data_struct, types)

True

In [229]:
builder_instance.config

WikitextConfig(name='wikitext-103-v1', version=1.0.0, data_dir=None, data_files=None, description='raw level dataset. The raw tokens before the addition of <unk> tokens. They should only be used for character level work or for creating newly derived datasets.')